In [1]:
import csv
import pandas as pd
import numpy as np
import pickle
from ast import literal_eval

In [8]:
movie_meta = pd.read_csv('all_movies_meta_wr_0418.csv')
movie_meta.shape

(856, 9)

In [5]:

with open("movies_meta_title_a1994_", "rb") as fp:   # Unpickling
    file_names = pickle.load(fp)
file_names = list(set(file_names))

In [9]:
# movie_meta.drop('release_year', inplace=True, axis=1)
cols = ['cast','director','genres','production_companies','keywords']
# # #changing columns into python objects
for c in cols:
    movie_meta[c] = movie_meta[c].apply(literal_eval)

In [10]:
cols = ['cast','director','genres','production_companies','keywords']
for c in cols:
    movie_meta[c + '_trans'] = movie_meta.apply(lambda x : ' '.join(x[c]), axis = 1)
movie_meta


,id,original_title,cast,director,genres,production_companies,keywords,release_year,imdb_id,cast_trans,director_trans,genres_trans,production_companies_trans,keywords_trans
0,333371,10 Cloverfield Lane,"[maryelizabethwinstead, johngoodman, johngalla...",[dantrachtenberg],"[thriller, sciencefiction, drama]","[paramountpictures, badrobot, spectrumeffects]","[kidnapping, bunker, paranoia]",2016.0,NaN,maryelizabethwinstead johngoodman johngallagherjr,dantrachtenberg,thriller sciencefiction drama,paramountpictures badrobot spectrumeffects,kidnapping bunker paranoia
1,4951,10 Things I Hate About You,"[juliastiles, heathledger, josephgordonlevitt]",[giljunger],"[comedy, romance, drama]","[madchance, jaretentertainment, touchstonepict...","[shakespeare, sister, highschool]",1999.0,tt0147800,juliastiles heathledger josephgordonlevitt,giljunger,comedy romance drama,madchance jaretentertainment touchstonepictures,shakespeare sister highschool
2,389,12 Angry Men,"[henryfonda, martinbalsam, johnfiedler]",[sidneylumet],[drama],"[unitedartists, orionnovaproductions]","[judge, jurors, sultriness]",1957.0,NaN,henryfonda martinbalsam johnfiedler,sidneylumet,drama,unitedartists orionnovaproductions,judge jurors sultriness
3,76203,12 Years a Slave,"[chiwetelejiofor, michaelfassbender, lupitanyo...",[stevemcqueen],"[drama, history]","[planbentertainment, regencyenterprises, river...","[whip, kidnapping, slavery]",2013.0,tt2024544,chiwetelejiofor michaelfassbender lupitanyongo,stevemcqueen,drama history,planbentertainment regencyenterprises riverroa...,whip kidnapping slavery
4,44115,127 Hours,"[jamesfranco, katemara, ambertamblyn]",[dannyboyle],"[adventure, drama, thriller]","[foxsearchlightpictures, duneentertainment, da...","[mountain, despair, adventure]",2010.0,tt1542344,jamesfranco katemara ambertamblyn,dannyboyle,adventure drama thriller,foxsearchlightpictures duneentertainment darlo...,mountain despair adventure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,310593,Youth,"[michaelcaine, harveykeitel, rachelweisz]",[paolosorrentino],"[comedy, drama]","[france2cinéma, indigofilm, pathé, numberfilms]","[venice, suicide, malenudity]",2015.0,NaN,michaelcaine harveykeitel rachelweisz,paolosorrentino,comedy drama,france2cinéma indigofilm pathé numberfilms,venice suicide malenudity
852,97630,Zero Dark Thirty,"[jessicachastain, jasonclarke, markstrong]",[kathrynbigelow],"[thriller, drama, history]","[columbiapictures, annapurnapictures, firstlig...","[assassination, cia, hotel]",2012.0,tt1790885,jessicachastain jasonclarke markstrong,kathrynbigelow,thriller drama history,columbiapictures annapurnapictures firstlightp...,assassination cia hotel
853,1949,Zodiac,"[jakegyllenhaal, robertdowneyjr, markruffalo]",[davidfincher],"[crime, drama, mystery]","[paramountpictures, warnerbros, phoenixpictures]","[california, sanfrancisco, killing]",2007.0,tt0443706,jakegyllenhaal robertdowneyjr markruffalo,davidfincher,crime drama mystery,paramountpictures warnerbros phoenixpictures,california sanfrancisco killing
854,19908,Zombieland,"[jesseeisenberg, woodyharrelson, emmastone]",[rubenfleischer],"[comedy, horror]",[columbiapictures],"[washingtondc, sistersisterrelationship, posta...",2009.0,tt1156398,jesseeisenberg woodyharrelson emmastone,rubenfleischer,comedy horror,columbiapictures,washingtondc sistersisterrelationship postapoc...


In [11]:
# for each feture cal sim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(stop_words='english')
features_items_matrix = []
features = ['original_title','keywords_trans','production_companies_trans','genres_trans','director_trans','cast_trans','release_year']
for f in features:
    if f == 'release_year':
        cosine_sim_matrix = []
        for i in range(len(movie_meta[f].tolist())):
            for m in range(len(movie_meta[f].tolist())):
                if i<m:
                    cosine_sim_matrix.append(1 - abs(movie_meta[f].tolist()[i] - movie_meta[f].tolist()[m]) / (movie_meta[f].tolist()[i] + movie_meta[f].tolist()[m]))
        features_items_matrix.append(np.array(cosine_sim_matrix))
    else:
        count_vec_matrix = count_vec.fit_transform(movie_meta[f])
        cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)
        features_items_matrix.append(cosine_sim_matrix[np.triu_indices_from(cosine_sim_matrix, k = 1)])
    #movies index mapping
    # mapping = pd.Series(movie_meta_a2014.index,index = movie_meta_a2014['original_title'])
    # np.savetxt(f + ".csv", cosine_sim_matrix, delimiter=",")
features_items_matrix







[array([0.33333333, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0.        , 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.33333333, 0.57735027, 0.40824829, ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.99576588, 0.98514976, 0.9992554 , ..., 0.99950199, 0.99776286,
        0.99826087])]

In [12]:
# f_weight[0]
print(len(features_items_matrix[6]))

365940


In [20]:
len(features_items_matrix)

7

In [93]:
f_weight

array([[0.1903835 , 0.3612997 , 0.1903835 , 0.47595874, 0.18213465,
        0.7140206 , 0.16563694]])

In [14]:
from sklearn import preprocessing
f_weight = [[0.004308,0.005656,0.002208,0.005,0.00777,0.00977,0.002008]]
# features = ['original_title','keywords_trans','production_companies_trans','genres_trans','director_trans','cast_trans','release_year']

f_weight = preprocessing.normalize(f_weight)
print(f_weight)

# print(cosine_sim_matrix.shape)
for w_index in range(len(f_weight[0])):
    features_items_matrix[w_index] = features_items_matrix[w_index] * f_weight[0][w_index]
print(len(features_items_matrix[0]))

[[0.27791679 0.36487868 0.14244203 0.32255894 0.5012566  0.63028018
  0.12953967]]
365940


In [16]:
# print(len(features_items_matrix[0]))
features_items_sim = np.array(features_items_matrix)
features_items_sim = features_items_sim.sum(axis=0)

In [88]:
#for test
test_m = [6,2,3]
ttt = np.ones((3, 3))
index = 0
for term in range(3):
    # count = 0
    
    for next_term in range(3):
        # count += 1
        if term == next_term or ttt[term][next_term] != 1:
            continue
        else:
            ttt[term][next_term] = test_m[index]
            ttt[next_term][term] = test_m[index]
            index += 1
print(ttt)


[[1. 6. 2.]
 [6. 1. 3.]
 [2. 3. 1.]]


In [17]:
# print(features_items_sim.shape)
#create ori sim matrix
# cs_m_len = features_items_sim.shape[0]
colla_sim_matrix = np.ones((856, 856))
index = 0
for term in range(856):
    # count = 0
    
    for next_term in range(856):
        # count += 1
        if term == next_term or colla_sim_matrix[term][next_term] != 1:
            continue
        else:
            colla_sim_matrix[term][next_term] = features_items_sim[index]
            colla_sim_matrix[next_term][term] = features_items_sim[index]
            index += 1
print(colla_sim_matrix)


[[1.         0.32914976 0.31384547 ... 0.2842502  0.12931439 0.12953967]
 [0.32914976 1.         0.31439387 ... 0.23680063 0.26090061 0.12899119]
 [0.31384547 0.31439387 1.         ... 0.31413521 0.12784122 0.12761598]
 ...
 [0.2842502  0.23680063 0.31413521 ... 1.         0.12947516 0.12924987]
 [0.12931439 0.26090061 0.12784122 ... 0.12947516 1.         0.12931439]
 [0.12953967 0.12899119 0.12761598 ... 0.12924987 0.12931439 1.        ]]


In [18]:
# colla_sim_matrix.shape
np.save('colla_sim_matrix_0423.npy', colla_sim_matrix) # save
r = np.load('colla_sim_matrix_0423.npy') # load
colla_sim_matrix == r
# np.savetxt("colla_sim_matrix.csv", colla_sim_matrix, delimiter=",")

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])